In [885]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import norm
from math import sqrt
import time 
import numpy as np
import matplotlib.animation as animation
%matplotlib qt
import pprint


In [886]:
N_t = 200
N_c = 200

w = np.zeros((N_c, N_t+1))

u = np.full_like(w, 1.0) # for now computing only one realization of the stochastic process


#readout_w_at_u_0 = ?

def show_animated_figure(u, cell_i = cell_i):
    fig = plt.figure()
    ylim_min = np.amin(u)
    ylim_max = np.amax(u)
    ax = plt.axes(xlim=(0, 210), ylim=(ylim_min, 2*ylim_max))
    fig.tight_layout()
    line, = ax.plot([], [], color='k', lw=2)

    def update(num, u, line):
        line.set_data(cell_i, u[:, num])
        return line,

    ani = animation.FuncAnimation(fig, update, N_t+1, fargs=[u, line],
                                  interval=25, blit=True)
    plt.show()
    
def save_animation(filename, animation_object):# supported format is mp4
    Writer = animation.writers['ffmpeg']
    writer = Writer(fps=15, metadata=dict(artist='Me'), bitrate=1800)
    animation_object.save(filename, writer=writer)

def comp_heaviside(cell_index, del_x, v_0, t):
    if (cell_index*del_x - v_0*t > 0):
        return 1
    else:
        return 0
    
def return_heaviside_matrix(N_c, N_t, del_x, v_0, dt):
    heaviside = np.zeros((N_c, N_t+1))
    for i in range (N_c):
        for j in range(N_t+1):
            heaviside[i][j] = int((i*del_x - v_0 * j* dt) > 0)
            #print (i*del_x - v_0 * j* dt)
    return heaviside
    
u0 = u[:, 0]
u0 = np.asarray(u0)
r_u = norm.rvs(size=u0.shape + (N_t,), scale=sigma**2)
r_w = norm.rvs(size=u0.shape + (N_t,), scale=sigma**2)



#### Get travelling heaviside function first

In [887]:
time_ = np.asarray([i for i in range(0, N_t)])
cell_i= np.asarray([i for i in range(0,N_c)])


In [888]:
tau_u =7 #min 
P_u = 1/tau_u# min^-1
v_0 = 2.55 # cdmin^-1
dt = tau_u/10

del_x = 1.5 #cd

tau_w = 20 # min
K_u = 5 
K_w = 1 
beta = 1e-250
sigma = 0.036**2 
P_w = 1/tau_w
n = 4
m =5



In [889]:
heaviside = return_heaviside_matrix(N_c, N_t-1, del_x, v_0, dt)

In [890]:
"""for i in range(1, N_t+1): ## for every time step update all cells 
    u[:, i] = u[:, i-1]*(1-1/tau_u)*dt + P_u*(np.heaviside(cell_i, v_0*time_))*dt #+ sqrt(dt)*r_u[:, i-1]
    if (i+1 <= N_t):
        if (i-tau_w < 0):
            w[:, i] = w[:, i-1]*(1 - 1/tau_w) + P_w*(u[:, i]**n/(K_u**n + u[:, i]**n))+beta*(w[:, i-1] + w[:, i+1]) + sqrt(dt)*r_w[:, i-1]
        else:
            w[:, i] = w[:, i-1]*(1 - 1/tau_w) + P_w*(u[:, i]**n/(K_u**n + u[:, i]**n))*(K_w**m/(K_w**m + w[:, i-tau_w]**m))+beta*(w[:, i-1] + w[:, i+1]) + sqrt(dt)*r_w[:, i-1]

    else:
        w[:, i] = w[:, i-1]*(1 - 1/tau_w) +  P_w*(u[:, i]**n/(K_u**n + u[:, i]**n))*(K_w**m/(K_w**m + w[:, i-tau_w]**m))+beta*(w[:, i-1]) + sqrt(dt)*r_w[:, i-1]
"""

'for i in range(1, N_t+1): ## for every time step update all cells \n    u[:, i] = u[:, i-1]*(1-1/tau_u)*dt + P_u*(np.heaviside(cell_i, v_0*time_))*dt #+ sqrt(dt)*r_u[:, i-1]\n    if (i+1 <= N_t):\n        if (i-tau_w < 0):\n            w[:, i] = w[:, i-1]*(1 - 1/tau_w) + P_w*(u[:, i]**n/(K_u**n + u[:, i]**n))+beta*(w[:, i-1] + w[:, i+1]) + sqrt(dt)*r_w[:, i-1]\n        else:\n            w[:, i] = w[:, i-1]*(1 - 1/tau_w) + P_w*(u[:, i]**n/(K_u**n + u[:, i]**n))*(K_w**m/(K_w**m + w[:, i-tau_w]**m))+beta*(w[:, i-1] + w[:, i+1]) + sqrt(dt)*r_w[:, i-1]\n\n    else:\n        w[:, i] = w[:, i-1]*(1 - 1/tau_w) +  P_w*(u[:, i]**n/(K_u**n + u[:, i]**n))*(K_w**m/(K_w**m + w[:, i-tau_w]**m))+beta*(w[:, i-1]) + sqrt(dt)*r_w[:, i-1]\n'

In [891]:
time_ = np.asarray([i*dt for i in range(0, N_t)])
cell_i= np.asarray([[i*del_x for j in range(0, N_t)] for i in range(0,N_c)])

In [892]:
cell_i.shape

(200, 200)

In [906]:
for i in range(1, N_t): 
    u[:, i] = (u[:,i-1] + P_u*(np.heaviside(cell_i[:, i], time_))*dt)/(1+dt/tau_u)

In [908]:
u

array([[1.00000000e+00, 9.09090909e-01, 8.26446281e-01, ...,
        6.37159758e-09, 5.79236144e-09, 1.00000000e+00],
       [1.00000000e+00, 1.00000000e+00, 1.00000000e+00, ...,
        1.00000000e+00, 1.00000000e+00, 1.00000000e+00],
       [1.00000000e+00, 1.00000000e+00, 1.00000000e+00, ...,
        1.00000000e+00, 1.00000000e+00, 1.00000000e+00],
       ...,
       [1.00000000e+00, 1.00000000e+00, 1.00000000e+00, ...,
        1.00000000e+00, 1.00000000e+00, 1.00000000e+00],
       [1.00000000e+00, 1.00000000e+00, 1.00000000e+00, ...,
        1.00000000e+00, 1.00000000e+00, 1.00000000e+00],
       [1.00000000e+00, 1.00000000e+00, 1.00000000e+00, ...,
        1.00000000e+00, 1.00000000e+00, 1.00000000e+00]])

In [909]:
fig = plt.figure()

ax = plt.axes(xlim=(0, 200), ylim=(-0.5*np.amin(u),2))
ax.set_ylabel("u")
ax.set_xlabel("cell_index_i")
fig.tight_layout()
line, = ax.plot([], [], color='steelblue', lw=2)

def update(num, u, line):
    line.set_data(time_, u[:, num])
    return line,

ani = animation.FuncAnimation(fig, update, N_t+1, fargs=[u, line],
                              interval=40, blit=True)
plt.show()

In [823]:
save_animation("u.mp4", ani)

In [319]:
plt.plot(time_, u[0, :])

In [65]:
"""for i in range(1, N+1): ## for every time step update all cells 
    u[:, i] = u[:, i-1]*(1 - 1/tau_u) + P_u*(1 - heaviside(i, del_x, v_0, i*dt)) + sqrt(dt)*r_u[:, i-1]
    if (i+1 <= N):
        w[:, i] = w[:, i-1]*(1 - 1/tau_w) + P_w*(u[:, i]**n/(K_u**n + u[:, i]**n))*(K_w**m/(K_w**m + w[:, i-tau_w]**m))
    
        +beta*(w[:, i-1] + w[:, i+1]) + sqrt(dt)*r_w[:, i-1]
    else:
        w[:, i] = w[:, i-1]*(1 - 1/tau_w) + P_w*(u[:, i]**n/(K_u**n + u[:, i]**n))*(K_w**m/(K_w**m + w[:, i-tau_w]**m))
    
        +beta*(w[:, i-1]) + sqrt(dt)*r_w[:, i-1]"""

'for i in range(1, N+1): ## for every time step update all cells \n    u[:, i] = u[:, i-1]*(1 - 1/tau_u) + P_u*(1 - heaviside(i, del_x, v_0, i*dt)) + sqrt(dt)*r_u[:, i-1]\n    if (i+1 <= N):\n        w[:, i] = w[:, i-1]*(1 - 1/tau_w) + P_w*(u[:, i]**n/(K_u**n + u[:, i]**n))*(K_w**m/(K_w**m + w[:, i-tau_w]**m))\n    \n        +beta*(w[:, i-1] + w[:, i+1]) + sqrt(dt)*r_w[:, i-1]\n    else:\n        w[:, i] = w[:, i-1]*(1 - 1/tau_w) + P_w*(u[:, i]**n/(K_u**n + u[:, i]**n))*(K_w**m/(K_w**m + w[:, i-tau_w]**m))\n    \n        +beta*(w[:, i-1]) + sqrt(dt)*r_w[:, i-1]'

In [71]:
heaviside.shape

(200, 101)

In [66]:
print (np.amin(u), np.amax(u))

0.0 0.9999997980141082
